In [5]:
import numpy as np
import itertools
from itertools import permutations
from itertools import combinations
from itertools import product
import matplotlib.pyplot as plt
import sympy as sp
import copy



#MesonParticle = [coefficient, [flavorstructure]]
pip = [1, [2.2, 1]]
pipB = [1, [1.2, 2]]
pi0 = [[1/(sp.sqrt(2)), [2.2, 2]], [-1/(sp.sqrt(2)), [1.2, 1]]]
pi0B = [[1/(sp.sqrt(2)), [2.2, 2]], [-1/(sp.sqrt(2)), [1.2, 1]]]
pim = [-1, [1.2, 2]]
pimB = [-1, [2.2, 1]]
eta0 = [[1, [2.2, 2]], [1, [1.2, 1]]]
eta0B = [[1, [2.2, 2]], [1, [1.2, 1]]]
kaonp = [1, [3.2, 1]]
kaonpB = [1, [1.2, 3]]
kaonm = [1, [3.2, 2]]
kaonmB = [1, [2.2, 3]]
phi0 = [1, [3.2, 3]]
phi0B = [1, [3.2, 3]]

#BaryonParticle = [coefficient, [flavorstructure]]
deltapp = [1, [1, 1, 1]]
deltappB = [1, [1.2, 1.2, 1.2]]
deltap = [sp.sqrt(3), [1, 1, 2]]
deltapB = [sp.sqrt(3), [2.2, 1.2, 1.2]]
delta0 = [sp.sqrt(3), [1, 2, 2]]
delta0B = [sp.sqrt(3), [2.2, 2.2, 1.2]]
deltam = [1, [2, 2, 2]]
deltamB = [1, [2.2, 2.2, 2.2]]
sigmap = [1, [1, 1, 3]]
sigmapB = [1, [3.2, 1.2, 1.2]]
sigma0 = [sp.sqrt(2), [1, 2, 3]]
sigma0B = [sp.sqrt(2), [3.2, 2.2, 1.2]]
sigmam = [1, [2, 2, 3]]
sigmamB = [1, [3.2, 2.2, 2.2]]
nucleonp = [1, [1, 1, 2]]
nucleonpB = [1, [2.2, 1.2, 1.2]]
nucleon0 = [-1, [2, 2, 1]]
nucleon0B = [-1, [1.2, 2.2, 2.2]]
xi0 = [1, [3, 3, 1]]
xi0B = [1, [1.2, 3.2, 3.2]]
xim = [1, [3, 3, 2]]
ximB = [1, [2.2, 3.2, 3.2]]
lambda0 = [1, [1, 2, 3]]
lambda0B = [1, [3.2, 2.2, 1.2]]
omegam = [1, [3, 3, 3]]
omegamB = [1, [3.2, 3.2, 3.2]]



Deltapp = ["d2", deltapp]
DeltappB = ["d2", deltappB]
Deltap = ["d1", deltap]
DeltapB = ["d1", deltapB]
Delta0 = ["d0", delta0]
Delta0B = ["d0", delta0B]
Deltam = ["d2", deltam]
DeltamB = ["d2", deltamB]
Sigmap = ["s1", sigmap]
SigmapB = ["s1", sigmapB]
Sigma0 = ["s0", sigma0]
Sigma0B = ["s0", sigma0B]
Sigmam = ["s1", sigmam]
SigmamB = ["s1", sigmamB]
Nucleonp = ["n", nucleonp]
NucleonpB = ["n", nucleonpB]
Nucleon0 = ["n", nucleon0]
Nucleon0B = ["n", nucleon0B]
Xi0 = ["x", xi0]
Xi0B = ["x", xi0B]
Xim = ["x", xim]
XimB = ["x", ximB]
Lambda0 = ["l", lambda0]
Lambda0B = ["l", lambda0B]
Omegam = ["o", omegam]
OmegamB = ["o", omegamB]


Pip = ["M", pip]
PipB = ["M", pipB]
Pi0 = ["M", pi0]
Pi0B = ["M", pi0B]
Pim = ["M", pim]
PimB = ["M", pimB]
Eta0 = ["M", eta0]
Eta0B = ["M", eta0B]
Kaonp = ["M", kaonp]
KaonpB = ["M", kaonpB]
Kaonm = ["M", kaonm]
KaonmB = ["M", kaonmB]
Phi0 = ["M", phi0]
Phi0B = ["M", phi0B]

In [ ]:
# provides the signs at the sink and source #
def sgn(number):
    if number > 0:
        return '+'
    elif number < 0:
        return '-'
    else:
        return '0'



##The quarks in a correlator are presented at some point as Permutation_0 = [[q0, 0],[q1, 1],[q2, 2],...].
##At that point, the numbers of quarks are extracted and given as a listm i.e. Permutation_0 -> [0, 1, 2..].
# The following function calculates the overall sign of a permutation based on the order of quark numbers in the list.
def permutation_sign(permuted_elements):
    inversions = 0
    nF = len(permuted_elements)
    for i in range(nF):
        for j in range(i + 1, nF):
            if permuted_elements[i] > permuted_elements[j]:
                inversions += 1
    sign = (-1) ** inversions
    return sign


## [x for inner_list in candidate for x in inner_list] creates a new list that consists of all the elements from the inner lists contained within the candidate list.
## Here, it is used to separate contracted quark pairs into individual quarks.
## set(map(tuple, all_elements)) converts each element in all_elements into a tuple, and then all the tuples are added to a set.
## A Wick contraction is considered valid if each unique quark in the correlator appears exactly once.
## The following function compares the length of the set consisting of initial quarks before contraction with the length of the set consisting of quarks after contraction.
# If the lengths are the same, the contraction is valid and the function gives True.
def is_valid_contraction(candidate, Start_List):
    all_elements = [elem1 for pairK in candidate for elem1 in pairK]
    return len(set(map(tuple, all_elements))) == len(Start_List)


## pairs=[list(pair) for pair in combinations(Start_List, 2)] generates a list where each element is a combination of two elements from Start_List. It includes all possible combinations while ignoring the order.
## paar_contraction has the form [[quarki, i], [quarkj, j]].
## In order for a contraction to be valid, quarki must be quark_j in bar (or other way). I.e. in our notation |quarki| = |quarkj| + 0.2 or  |quarki| + 0.2 = |quarkj|
## So we exclude all pairs, which do not satisfy that condition! All possible pairs are saved in "possible_pairs"
## We always must have an even number of quarks in the correlators, let's say 2N. 
## We now create the list pairs2 from possible_pairs. Each element of it is a combination of N elements from possible_pairs, meaning that each element in pairs2 could be a valid Wick contraction candidate.
# The following function checks the validity of each element in pairs2 using the previous function and removes the invalid ones.
def Generate_Contraction(Start_List):
    pairs = [list(pair) for pair in combinations(Start_List, 2)]
    possible_pairs = []
    for paar_contraction in pairs:
        if np.abs(paar_contraction[0][0]) == (np.abs(paar_contraction[1][0]) + 0.2) or (np.abs(paar_contraction[0][0]) + 0.2) == np.abs(paar_contraction[1][0]):
            possible_pairs.append(paar_contraction)
    pairs2 = [list(pairX) for pairX in combinations(possible_pairs, len(Start_List) // 2)]
    final_solution = []
    for candidate in pairs2:
        if is_valid_contraction(candidate, Start_List):
            final_solution.append(candidate)
    return final_solution

In [ ]:
## The function Correlator(Sink, Source) takes two lists of quarks, one at the sink and one at the source
## the list at the sink and source are of the form:  [hadron_1, hadron_2,...]. where hadron_i can be e.g. Nucleonp or NucleonpB.
# Correlator(Sink, Source) plots as an output the quarks at the Sink and Source with numbering at it gives all possible Wick contractions with the correpsonding numerical factor for each diagram.
def Correlator(Sink, Source):

    lSink = len(Sink)
    lSource = len(Source)

#z0 corresponds to the overall constant.
    z0 = 1
###
    for i in range(lSink):
        z0 *= Sink[i][0]
    for j in range(lSource):
        z0 *= Source[j][0]
###
    total_operator = []
    source_plot = []
    sink_plot = []

#### Now combine all quark operators into one operator, i.e. in the same list ####
# The idea is to take all quarks form the sink list and put them into one list, where each quark is multiplied with a number that has all informations about its indices.
# quarks at the sink are multiplied with minus. So if we have 2 quarks at the sink and 2 at the sink, we have at the end in "f_t_o" the following list [[-qsink1, 0],[-qsink1, 1], [qsource0, 0], [qsource1, 1]]
    counter = 0

    for i in range(lSink):
        sub_list = []
        for quark in Sink[i][1]:
            sub_list.append([-1 * quark, counter])
            counter += 1
        total_operator.append(sub_list)
        sink_plot.append(sub_list)
    max_counter1 = counter
    counter = 0

    for j in range(lSource):
        sub_list = []
        for quark in Source[j][1]:
            sub_list.append([quark, counter])
            counter += 1
        total_operator.append(sub_list)
        source_plot.append(sub_list)
    max_counter2 = counter


    sinkplot = [element for subliste in sink_plot for element in subliste]
    sourceplot = [element for subliste in source_plot for element in subliste]

    f_t_o = [quark for hadron in total_operator for quark in hadron]
    lf_t_o = len(f_t_o)
    first_Step = Generate_Contraction(f_t_o)

#Now we put all quarks in bar on the right hand side (so it corresponds to the definition of the propagator in two pair contraction)

    second_Step = []
    for diagram in first_Step:
        new_diagram = []
        for paar_contraction in diagram:
            if (np.abs(paar_contraction[0][0]) + 0.2) == np.abs(paar_contraction[1][0]):
                new_diagram.append(paar_contraction)
            else:
                new_diagram.append([paar_contraction[1], paar_contraction[0]])
        second_Step.append(new_diagram)

# We create now a list which contains the signs coming form the Grassmann-Algebra
    sign_of_diagram = [0 for _ in second_Step]
    diagram_N = 0
    m_c = max_counter1 + 1

    for diagram in second_Step:
        numbers = []

        for paar_contraction in diagram:
            if paar_contraction[0][0] < 0:
                t1 = paar_contraction[0][1]
            else:
                t1 = paar_contraction[0][1] + m_c
            numbers.append(t1)
            if paar_contraction[1][0] < 0:
                t2 = paar_contraction[1][1]
            else:
                t2 = paar_contraction[1][1] + m_c
            numbers.append(t2)

        sign_of_diagram[diagram_N] = permutation_sign(numbers)
        diagram_N += 1
        
        
    
# The Wick contraction diagarms are saved in "last_step" in the following form: [[numerical_factor_of_diagram_0, Diagram0], [numerical_factor_of_diagram_1, Diagram1], ...]

    last_step = [0 for _ in second_Step]
    for i, diagram in enumerate(second_Step):
        Diagram_Propagator_Form = [0 for _ in diagram]
        for j, pair_contraction in enumerate(diagram):
            Diagram_Propagator_Form[j] = pair_contraction
        last_step[i] = [sign_of_diagram[i] * z0, Diagram_Propagator_Form]

#Do the plots now

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.axis('off')
    ax1.set_title("Sink", loc='left')
    shift1 = 0
    shift2 = 0
    for quark in sinkplot:
        if np.abs(quark[0]) == 1:
            ax1.text(0.07, 0.8-shift1, fr'$u(-{quark[1]})$', fontsize=13, ha='center')
            shift1 += 0.1
        if np.abs(quark[0]) == 1.2:
            ax1.text(0.07, 0.8-shift1, fr'$\overline{{u}}(-{quark[1]})$', fontsize=13, ha='center')
            shift1 += 0.1
        if np.abs(quark[0]) == 2:
            ax1.text(0.07, 0.8-shift1, fr'$d(-{quark[1]})$', fontsize=13, ha='center')
            shift1 += 0.1
        if np.abs(quark[0]) == 2.2:
            ax1.text(0.07, 0.8-shift1, fr'$\overline{{d}}(-{quark[1]})$', fontsize=13, ha='center')
            shift1 += 0.1
        if np.abs(quark[0]) == 3:
            ax1.text(0.07, 0.8-shift1, fr'$s(-{quark[1]})$', fontsize=13, ha='center')
            shift1 += 0.1
        if np.abs(quark[0]) == 3.2:
            ax1.text(0.07, 0.8-shift1, fr'$\overline{{s}}(-{quark[1]})$', fontsize=13, ha='center')
            shift1 += 0.1
    ax2.axis('off')
    ax2.set_title("Source", loc='left')
    for quark in sourceplot:
        if np.abs(quark[0]) == 1:
            ax2.text(0.1, 0.8-shift2, fr'$u({quark[1]})$', fontsize=13, ha='center')
            shift2 += 0.1
        if np.abs(quark[0]) == 1.2:
            ax2.text(0.1, 0.8-shift2, fr'$\overline{{u}}({quark[1]})$', fontsize=13, ha='center')
            shift2 += 0.1
        if np.abs(quark[0]) == 2:
            ax2.text(0.1, 0.8-shift2, fr'$d({quark[1]})$', fontsize=13, ha='center')
            shift2 += 0.1
        if np.abs(quark[0]) == 2.2:
            ax2.text(0.1, 0.8-shift2, fr'$\overline{{d}}({quark[1]})$', fontsize=13, ha='center')
            shift2 += 0.1
        if np.abs(quark[0]) == 3:
            ax2.text(0.1, 0.8-shift2, fr'$s({quark[1]})$', fontsize=13, ha='center')
            shift2 += 0.1
        if np.abs(quark[0]) == 3.2:
            ax2.text(0.1, 0.8-shift2, fr'$\overline{{s}}({quark[1]})$', fontsize=13, ha='center')
            shift2 += 0.1
    plt.show()

    return last_step